In [11]:
%load_ext autoreload
%autoreload 2

In [4]:
data_root = "../../../../../../Desktop/DevLongEvalTrainCollection/"
! tree {data_root} -L 3

../../../../../../Desktop/DevLongEvalTrainCollection/
├── 2023_01
│   ├── English
│   │   ├── Documents
│   │   ├── Qrels
│   │   └── Queries
│   ├── French
│   │   ├── Documents
│   │   ├── Qrels
│   │   └── Queries
│   └── count_docs.sh
└── _SUCCESS

10 directories, 2 files


In [1]:
import os
os.environ['SPARK_LOCAL_DIR'] = ''
print(os.environ.get('SPARK_LOCAL_DIR'))

/Users/ikafrulbasha/Library/Python/3.9/lib/python/site-packages/pyspark


In [3]:
from longeval.spark import get_spark

spark = get_spark()
spark

25/01/18 12:41:39 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [5]:
from longeval.collection import RawCollection

train_eng = RawCollection(
    spark, f"{data_root}/2023_01/English"
)
docs = train_eng.documents.cache()
%count docs.count()

UsageError: Line magic function `%count` not found.                             


In [6]:
docs.show(5, vertical=True, truncate=80)

-RECORD 0------------------------------------------------------------------------------------
 contents | Forget the Nukes:\nTunnels\nAre North\nKorea's Secret Weapon\n|\nThe\nNationa... 
 docid    | doc012310000001                                                                  
-RECORD 1------------------------------------------------------------------------------------
 contents | Project Perfect Mod ::\nView topic\n- Who will take over the island?\nDo you ... 
 docid    | doc012310000002                                                                  
-RECORD 2------------------------------------------------------------------------------------
 contents | "Lawyer James B. Donovan, widely known for negotiating the release of...\nNie... 
 docid    | doc012310000003                                                                  
-RECORD 3------------------------------------------------------------------------------------
 contents | What Rex Tillerson'\ns Nomination Means for Russ

In [7]:
train_eng.queries.show(5, vertical=True)

-RECORD 0---------------------
 qid   | q012318              
 query | case over the border 
-RECORD 1---------------------
 qid   | q012396              
 query | water atlantic       
-RECORD 2---------------------
 qid   | q0123180             
 query | blanquette de vea... 
-RECORD 3---------------------
 qid   | q0123240             
 query | gift woman           
-RECORD 4---------------------
 qid   | q0123387             
 query | Government           
only showing top 5 rows



In [8]:
train_eng.qrels.show(5, vertical=True)

-RECORD 0----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012303114898 
 rel   | 0               
-RECORD 1----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012307806130 
 rel   | 1               
-RECORD 2----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012311314092 
 rel   | 0               
-RECORD 3----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012301310209 
 rel   | 0               
-RECORD 4----------------
 qid   | q012318         
 rank  | 0               
 docid | doc012311608989 
 rel   | 0               
only showing top 5 rows



In [12]:
from opensearchpy import OpenSearch

client = OpenSearch(
    hosts="http://localhost:9200",
)
client.info()

{'name': 'ec8647ef7bdf',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'ZfUnTr8ZT-Ov2PK_CobhPw',
 'version': {'distribution': 'opensearch',
  'number': '2.18.0',
  'build_type': 'tar',
  'build_hash': '99a9a81da366173b0c2b963b26ea92e15ef34547',
  'build_date': '2024-10-31T19:08:04.231254959Z',
  'build_snapshot': False,
  'lucene_version': '9.12.0',
  'minimum_wire_compatibility_version': '7.10.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'The OpenSearch Project: https://opensearch.org/'}

In [13]:
# add a default index template that sets replicas to 0
# also make an indexable contents and docid fields
client.indices.put_template(
    name="default",
    body={
        "index_patterns": ["*"],
        "settings": {
            "number_of_replicas": 0,
            "number_of_shards": 4,
        },
        "mappings": {
            "properties": {
                "contents": {
                    "type": "text",
                },
                "docid": {
                    "type": "keyword",
                },
            },
        },
    },
)

{'acknowledged': True}

In [14]:
# list all indices
print(client.cat.indices())

green open .opensearch-observability NjP0f8pEQRWGF5Dzevrkbw 1 0 0 0  208b  208b
green open .plugins-ml-config        p4PGo4QqTui16JhBHpaqvQ 1 0 1 0   4kb   4kb
green open .ql-datasources           t64TwkMFS9WIVqms2plQvA 1 0 0 0  208b  208b
green open .kibana_1                 FP__coYdQUCWeAtADNY-iA 1 0 1 0 5.3kb 5.3kb



In [ ]:
from longeval.collection import ParquetCollection
from longeval.spark import get_spark

spark = get_spark()

root = "../../data/longeval/parquet/"
collection = ParquetCollection(spark, f"{root}/2023_01/Documents/Json")
collection.documents.show(n=5)

In [20]:
index_name = "test_index_1"  

document = {
    "contents": "This is a test document.",
    "docid": "12345"
}

response = client.index(
    index=index_name,
    body=document,
    id="1"  
)

In [21]:
response = client.indices.get_mapping(index=index_name)
print(response)

{'test_index_1': {'mappings': {'properties': {'contents': {'type': 'text'}, 'docid': {'type': 'keyword'}}}}}


In [22]:
index_name = "test_index_1"  

document = {
    "contents": "This is the 2nd test document.",
    "docid": "111"
}

response = client.index(
    index=index_name,
    body=document,
    id="2"  
)

In [23]:
index_name = "animal_index"  

doc_1 = {
    "contents": "I am a dog. I like the color green.",
    "docid": "111"
}

doc_2 = {
    "contents": "I am a cat. I like the color purple but sometimes red.",
    "docid": "222"
}

doc_3 = {
    "contents": "I am a horse. I dont like the color orange but I like blue.",
    "docid": "333"
}

response = client.index(
    index=index_name,
    body=doc_1,
    id="1"  
)

response = client.index(
    index=index_name,
    body=doc_2,
    id="2"  
)

response = client.index(
    index=index_name,
    body=doc_3,
    id="3"  
)

In [25]:
from opensearchpy import OpenSearch

query = {
    "query": {
        "match": {
            "contents": "like" # appears twice in doc 3
        }
    }
}

response = client.search(
    index="animal_index",  
    body=query             
)

print(response) # max score matches the horse statement

{'took': 9, 'timed_out': False, '_shards': {'total': 4, 'successful': 4, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 0.17382786, 'hits': [{'_index': 'animal_index', '_id': '3', '_score': 0.17382786, '_source': {'contents': 'I am a horse. I dont like the color orange but I like blue.', 'docid': '333'}}, {'_index': 'animal_index', '_id': '1', '_score': 0.1473054, '_source': {'contents': 'I am a dog. I like the color green.', 'docid': '111'}}, {'_index': 'animal_index', '_id': '2', '_score': 0.13198866, '_source': {'contents': 'I am a cat. I like the color purple but sometimes red.', 'docid': '222'}}]}}
